In [47]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf

import sys
sys.path.append('/mnt/c/Users/kheut/code/covid19-forecasting/tf_model_1p5/')

from enum import Enum

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense

import tensorflow_probability as tfp
from scipy.stats import beta, truncnorm


# Local imports from model.py, data.py
from model import CovidModel, LogPoissonProb, get_logging_callbacks, Comp, Vax
from data import read_data, create_warmup
#from plots import make_all_plots

import scipy

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 20}) # set plot font sizes

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [48]:
transition_window =3

warmup_start = '20210428'
warmup_end = '20210430'
train_start = '20210501'
train_end = '20210731'
test_start = '20210801'
test_end = '20210831'

state = 'Massachusetts'
state_abbrev = 'MA'

data_dir = '../data'
covid_estim_date = '20210901'
hhs_date = '20210903'
owid_date = '20210903'

log_dir = './logs/new_warmup'


# Learning rate
model_learning_rate = 1e-2
warmup_learning_rate = 20

In [49]:
df = read_data(data_dir=data_dir,
               covid_estim_date=covid_estim_date,
               hhs_date=hhs_date,
               owid_date=owid_date,
               state=state, state_abbrev=state_abbrev)

In [50]:
class Comp(Enum):
    A = 0
    M = 1
    #X = 2
    #G = 3
    
class Vax(Enum):
    total = -1
    no = 0
    yes = 1

In [51]:
# get warmup arrays, splitting on vaccination status
warmup_asymp, warmup_mild, warmup_extreme = create_warmup(df, 
                                                          warmup_start, 
                                                          warmup_end,
                                                          0,0,0)

# re-combine
warmup_asymp = warmup_asymp[Vax.no.value] + warmup_asymp[Vax.yes.value]

In [75]:
synth_T_serial = 5.8
synth_rho_M = 0.76
synth_lambda_M = 4.7
synth_nu_M = 3.1

In [76]:
warmup_A_params = {}
warmup_A_params[Vax.total.value] = {}
warmup_A_params[Vax.total.value]['prior'] = []
warmup_A_params[Vax.total.value]['posterior_init'] = []

for day in range(transition_window):
    warmup_A_params[Vax.total.value]['prior'].append({'loc': warmup_asymp[day],
                                                'scale': warmup_asymp[day]/10})
    

In [77]:
x_train = tf.cast(df.loc[train_start:test_end,'Rt'].values, dtype=tf.float32)
#y_test = tf.cast(df.loc[train_start:test_end,'mild'], dtype=tf.float32)

In [78]:
A_vals = []
M_vals =[]
for day in range(transition_window):
    A_vals.append(warmup_A_params[-1]['prior'][day]['loc'])

for day in range(len(x_train)):
    yesterday_asymp = A_vals[-1]
    
    today_asymp = yesterday_asymp*x_train[day]**(1/synth_T_serial)
    A_vals.append(today_asymp)
    
    today_M = 0
    pi_M=[]
    for j in range(transition_window):
        
        
        lambda_M_fix = synth_lambda_M
        nu_M_fix = synth_nu_M
        poisson_dist = scipy.stats.poisson(lambda_M_fix)
        pi_M_j_ago = poisson_dist.logpmf(j+1)/nu_M_fix
        
        pi_M.append(pi_M_j_ago)
    
    pi_M = scipy.special.softmax(pi_M)
        
    for j in range(transition_window):
        j_ago_asymp = A_vals[day-j-1]
        today_M += j_ago_asymp*synth_rho_M*pi_M[j]
        
    M_vals.append(today_M)

In [79]:

T_serial = {}
T_serial[Vax.total.value] = {}
T_serial[Vax.total.value]['prior'] ={'loc':5.8, 'scale':1}


rho_M = {}
rho_M[Vax.total.value] = {}
rho_M[Vax.total.value]['prior'] = {'a': 31.8, 'b': 10.3}

lambda_M = {}
lambda_M[Vax.total.value] = {}
lambda_M[Vax.total.value]['prior'] = {'loc': 4.7, 'scale': 1}

nu_M = {}
nu_M[Vax.total.value] = {}
nu_M[Vax.total.value]['prior'] = {'loc': 3.1, 'scale': 1.2}




In [80]:
T_serial_scale = 1.0
rho_M_scale = 0.1
lambda_M_scale = 1.0
nu_M_scale = 1.2

warmup_scales = [0.1]*warmup_asymp

In [140]:
T_serial[Vax.total.value]['posterior_init'] = {'loc': tfp.math.softplus_inverse(4.0),
                                     'scale':tf.cast(tfp.math.softplus_inverse(T_serial_scale),dtype=tf.float32)}

rho_M[Vax.total.value]['posterior_init'] = {'loc': tf.cast(np.log(0.5/(1-0.5)),dtype=tf.float32),
                                      'scale':tf.cast(tfp.math.softplus_inverse(rho_M_scale),dtype=tf.float32)}

lambda_M[Vax.total.value]['posterior_init'] = {'loc': tf.cast(tfp.math.softplus_inverse(3.0),dtype=tf.float32),
                                         'scale':tf.cast(tfp.math.softplus_inverse(lambda_M_scale),dtype=tf.float32)}

nu_M[Vax.total.value]['posterior_init'] = {'loc': tf.cast(tfp.math.softplus_inverse(5.0),dtype=tf.float32),
                                     'scale':tf.cast(tfp.math.softplus_inverse(nu_M_scale),dtype=tf.float32)}

for day in range(transition_window):
    # must be positive so reverse softplus the mean
    warmup_A_params[Vax.total.value]['posterior_init'].append({'loc': tf.cast(tfp.math.softplus_inverse(2000.0),dtype=tf.float32),
                                                         'scale': tf.cast(tfp.math.softplus_inverse(500.0),dtype=tf.float32)})#tf.cast(tfp.math.softplus_inverse(warmup_asymp[day]/10),dtype=tf.float32)})

model = CovidModel([Vax.total], [Comp.A, Comp.M],
                 transition_window,
                T_serial, rho_M, lambda_M, nu_M,
                 warmup_A_params, posterior_samples=1000, debug_disable_theta=False)

In [141]:
model.variables

[<tf.Variable 'T_serial_A_loc_-1:0' shape=() dtype=float32, numpy=3.9815147>,
 <tf.Variable 'T_serial_A_scale_-1:0' shape=() dtype=float32, numpy=0.54132485>,
 <tf.Variable 'rho_M_loc_-1:0' shape=() dtype=float32, numpy=0.0>,
 <tf.Variable 'rho_M_scale_-1:0' shape=() dtype=float32, numpy=-2.2521684>,
 <tf.Variable 'lambda_M_loc_-1:0' shape=() dtype=float32, numpy=2.9489307>,
 <tf.Variable 'lambda_M_scale_-1:0' shape=() dtype=float32, numpy=0.54132485>,
 <tf.Variable 'nu_M_loc_-1:0' shape=() dtype=float32, numpy=4.9932394>,
 <tf.Variable 'nu_M_scale_-1:0' shape=() dtype=float32, numpy=0.8416177>,
 <tf.Variable 'warmup_A_loc_0_-1:0' shape=() dtype=float32, numpy=2000.0>,
 <tf.Variable 'warmup_A_scale_0_-1:0' shape=() dtype=float32, numpy=500.0>,
 <tf.Variable 'warmup_A_loc_1_-1:0' shape=() dtype=float32, numpy=2000.0>,
 <tf.Variable 'warmup_A_scale_1_-1:0' shape=() dtype=float32, numpy=500.0>,
 <tf.Variable 'warmup_A_loc_2_-1:0' shape=() dtype=float32, numpy=2000.0>,
 <tf.Variable 'warmu

In [142]:
warmup_variable_idx = -6
model_variables = model.variables[:warmup_variable_idx]
assert all(['warmup' not in variable.name for variable in model_variables])
warmup_variables = model.variables[warmup_variable_idx:]
assert all(['warmup' in variable.name for variable in warmup_variables])

In [143]:
loss = LogPoissonProb() 
optimizer = tf.keras.optimizers.SGD(
    learning_rate=0.001,# beta_1=0.1, beta_2=0.1
)

In [144]:
logging_callbacks = get_logging_callbacks('/mnt/c/Users/kheut/logs/covid/bayes_learn_all_custom_01')

2021-12-06 13:02:32.383183: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-06 13:02:32.383213: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-06 13:02:32.383285: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1666] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.
2021-12-06 13:02:32.383492: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-06 13:02:32.383541: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1757] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.


In [145]:
epochs = 1000
logs = {}

#model.compile(loss=loss, optimizer=optimizer, run_eagerly=True)
callbacks = tf.keras.callbacks.CallbackList(logging_callbacks, add_history=False, model=model)

for epoch in range(epochs):
    callbacks.on_epoch_begin(epoch, logs=logs)
    #callbacks.on_batch_begin(0, logs=logs)
   
    with tf.GradientTape() as tape:
        
        loss_val = loss(M_vals, model(x_train, training=True))
        
        
    grads = tape.gradient(loss_val, model.trainable_variables)
        
    grads[:warmup_variable_idx] = [grad*model_learning_rate if grad is not None else grad for grad in grads[:warmup_variable_idx] ]
    grads[warmup_variable_idx:] = [grad*warmup_learning_rate if grad is not None else grad for grad in grads[warmup_variable_idx:] ]

    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    

    callbacks.on_batch_end(0, logs=logs)
    
    callbacks.on_epoch_end(epoch, logs=logs)

        
callbacks.on_train_end(logs=logs)

KeyboardInterrupt: 

In [137]:
grads

[<tf.Tensor: shape=(), dtype=float32, numpy=-44.057255>,
 <tf.Tensor: shape=(), dtype=float32, numpy=20.409847>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-59.42387>,
 <tf.Tensor: shape=(), dtype=float32, numpy=1.1684586>,
 None,
 None,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.014274821>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.0052253297>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-1.2161665>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.45110703>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-2.4874573>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.8034713>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-343.2962>,
 <tf.Tensor: shape=(), dtype=float32, numpy=385.70175>]

In [107]:
model.compile(loss=loss, optimizer=optimizer, run_eagerly=True)
model.fit(x=np.asarray([x_train]), y=np.asarray([M_vals]),
         epochs=1000, batch_size=0,
        callbacks=logging_callbacks)

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 34680.5508
Epoch 2/1000


2021-12-06 12:52:30.203353: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2021-12-06 12:52:30.203378: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2021-12-06 12:52:30.203533: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1666] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI could not be loaded or symbol could not be found.


1/1 [==============================] - ETA: 0s - loss: 12128.7979

2021-12-06 12:52:32.777944: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-12-06 12:52:32.783832: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1757] function cupti_interface_->Finalize()failed with error CUPTI could not be loaded or symbol could not be found.
2021-12-06 12:52:32.949888: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:673]  GpuTracer has collected 0 callback api events and 0 activity events. 
2021-12-06 12:52:33.274078: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2021-12-06 12:52:34.167760: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: /mnt/c/Users/kheut/logs/covid/bayes_learn_all_custom_01/train/plugins/profile/2021_12_06_12_52_33

2021-12-06 12:52:34.889705: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to /mnt/c/Users/kheut/logs/covid/bayes_learn_all_custom_01/train/plugin

1/1 [==============================] - 5s 5s/step - loss: 12128.7979


2021-12-06 12:52:35.156247: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: /mnt/c/Users/kheut/logs/covid/bayes_learn_all_custom_01/train/plugins/profile/2021_12_06_12_52_33

2021-12-06 12:52:35.158122: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to /mnt/c/Users/kheut/logs/covid/bayes_learn_all_custom_01/train/plugins/profile/2021_12_06_12_52_33/MrChipsNVME.memory_profile.json.gz
2021-12-06 12:52:35.176211: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: /mnt/c/Users/kheut/logs/covid/bayes_learn_all_custom_01/train/plugins/profile/2021_12_06_12_52_33
Dumped tool data for xplane.pb to /mnt/c/Users/kheut/logs/covid/bayes_learn_all_custom_01/train/plugins/profile/2021_12_06_12_52_33/MrChipsNVME.xplane.pb
Dumped tool data for overview_page.pb to /mnt/c/Users/kheut/logs/covid/bayes_learn_all_custom_01/train/plugins/profile/2021_12_06_12_52_33/MrChipsNVME.overvi

Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 9947.0215
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 8190.0591
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 8202.5781
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 8850.3242
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 7987.3457
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 7889.0884
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 8263.8242
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 8168.0312
Epoch 11/1000
1/1 [==============================] - 2s 2s/step - loss: 7914.8687
Epoch 12/1000
1/1 [==============================] - 2s 2s/step - loss: 7909.8052
Epoch 13/1000
1/1 [==============================] - 2s 2s/step - loss: 7712.4424
Epoch 14/1000
1/1 [==============================] - 2s 2s/step - loss: 8444.6953
Epoch 15/1000
1/1 [====

KeyboardInterrupt: 

In [88]:
epoch

0

In [23]:
import tfa.optimizers

ModuleNotFoundError: No module named 'tfa'

In [40]:
tape.gradient(loss_val, model_variables)

RuntimeError: A non-persistent GradientTape can only be used to compute one set of gradients (or jacobians)

In [108]:
model.trainable_variables

[<tf.Variable 'T_serial_A_loc_-1:0' shape=() dtype=float32, numpy=5.8847>,
 <tf.Variable 'T_serial_A_scale_-1:0' shape=() dtype=float32, numpy=-3.0389266>,
 <tf.Variable 'rho_M_loc_-1:0' shape=() dtype=float32, numpy=27.03185>,
 <tf.Variable 'rho_M_scale_-1:0' shape=() dtype=float32, numpy=-2.5363524>,
 <tf.Variable 'lambda_M_loc_-1:0' shape=() dtype=float32, numpy=3.0260434>,
 <tf.Variable 'lambda_M_scale_-1:0' shape=() dtype=float32, numpy=0.5457453>,
 <tf.Variable 'nu_M_loc_-1:0' shape=() dtype=float32, numpy=4.8226805>,
 <tf.Variable 'nu_M_scale_-1:0' shape=() dtype=float32, numpy=0.87884235>,
 <tf.Variable 'warmup_A_loc_0_-1:0' shape=() dtype=float32, numpy=2000.0001>,
 <tf.Variable 'warmup_A_scale_0_-1:0' shape=() dtype=float32, numpy=499.99854>,
 <tf.Variable 'warmup_A_loc_1_-1:0' shape=() dtype=float32, numpy=2000.0002>,
 <tf.Variable 'warmup_A_scale_1_-1:0' shape=() dtype=float32, numpy=499.99704>,
 <tf.Variable 'warmup_A_loc_2_-1:0' shape=() dtype=float32, numpy=1999.8344>,
 

In [109]:
grads

[<tf.Tensor: shape=(), dtype=float32, numpy=-5126.562>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2788.328>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-27485.785>,
 <tf.Tensor: shape=(), dtype=float32, numpy=215.74144>,
 None,
 None,
 <tf.Tensor: shape=(), dtype=float32, numpy=-1.8661408>,
 <tf.Tensor: shape=(), dtype=float32, numpy=1.0120174>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.11260508>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.014422836>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-0.22747979>,
 <tf.Tensor: shape=(), dtype=float32, numpy=0.03405691>,
 <tf.Tensor: shape=(), dtype=float32, numpy=-30.666119>,
 <tf.Tensor: shape=(), dtype=float32, numpy=14.22691>]

In [139]:
loss_val

<tf.Tensor: shape=(), dtype=float32, numpy=12470.049>